# Friedmannove jednadžbe

Ovo je prilagodba radnog listića dostupnog [ovdje](http://sagemanifolds.obspm.fr/examples.html). 

Da bi sve funkcioniralo, osim jupyter notebooka potrebno je instalirati i Sage kernel. To se na Arch linuxu može tim redom. Drugdje je možda pogodnije naprosto instalirati Sage koji mislim dolazi sa svojim jupyter notebookom.

In [ ]:
version()

Postavljanje prikaza simbola korištenjem LaTeX formatiranja:

In [ ]:
%display latex

Deklariramo prostorvrijeme kao 4-dimenzionalnu diferencijabilnu mnogostrukost $M$:

In [ ]:
M = Manifold(4, 'M')
print(M)

Uvodimo standardne (FL)RW koordinate, koristeći metodu `chart()`, s argumentom koji je python string s razmaknuto navedenim koordinatama u sintaksi
`simbol:raspon vrijednosti:LaTeX simbol`
gdje je defaultni raspon vrijednosti $(-\infty, \infty)$:

In [ ]:
fr.<t,r,th,ph> = M.chart(r't r:[0,+oo) th:[0,pi]:\theta ph:[0,2*pi):\phi')
fr

Definiramo skalarne varijable: Newtonovu konstantu $G$, konstantu prostorne zakrivljenosti $k\in\{1,0,-1\}$, radijus zakrivljenosti $R_{0}$, faktor skale $a(t)$, gustoća kozmičkog fluida $\rho(t)$ i njegov tlak $p(t)$. Za RW metriku koristimo izraz iz Ryden (2016)

$$ ds^2 = c^2 dt^2 - a^{2}(t)\left(\frac{dr^2}{1-\frac{k r^2}{R_{0}^2}} + r^2 d\Omega\right) $$
  
uz napomenu da koristimo suprotni preznak metrike i da većina literature apsorbira radijus zakrivljenosti $R_0$ ili u konstantu zakrivljenosti $K \equiv k/R_{0}^2$ ili u faktor skale uz redefiniciju koordinate $r\to r R_0$ tako da postane bezdimenzionalna.

In [ ]:
var('G, Lambda, k, R0, c', domain='real')
a = M.scalar_field(function('a')(t), name='a')
rho = M.scalar_field(function('rho')(t), name='rho')
p = M.scalar_field(function('p')(t), name='p')

RW metriku definiramo u "-" konvenciji (tj. "čestičarska" ili "West coast", dakle obrnuto od Ryden):

In [ ]:
g = M.lorentzian_metric('g', signature='negative')
g[0,0] = c*c
g[1,1] = -a*a/(1 - k*r^2/R0^2)
g[2,2] = -a*a*r^2
g[3,3] = -a*a*(r*sin(th))^2
g.display()

<p>A matrix view of the metric components:</p>

In [ ]:
g[:]

<p>Christoffelovi simboli su kvadratni u metričkom tenzoru pa ne ovise o konvencijama za predznake:</p>

In [ ]:
nabla = g.connection()
g.christoffel_symbols_display()

Riccijev tenzor:

In [ ]:
Ricci = nabla.ricci()
Ricci.display_comp()

Ovi izrazrazi su, nakon zamjene $k \to k R_0$, u slaganju s [wikipedijom](https://en.wikipedia.org/wiki/Friedmann%E2%80%93Lema%C3%AEtre%E2%80%93Robertson%E2%80%93Walker_metric#Spherical_coordinates):

$$ R_{tt} = - 3 \frac{\ddot{a}}{a}, \\
R_{rr}=\frac{c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k}{1 - kr^2} \\
R_{\theta\theta} = r^2(c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k) \\
R_{\phi\phi} =r^2(c^{-2}(a(t)\ddot{a}(t) + 2\dot{a}^2(t)) + 2k)\sin^2(\theta)$$

Iz dokumentacije SageManifolds paketa može se zaključiti da isti koristi "+" konvenciju za prezdnak Riemannovog tenzora, kao i "+" konvenciju za vezu Riccijevog i Riemannovog tenzora (gdje za razliku od gornje konvencije za metriku korisnik to ne može mijenjati), što znači da je nužno pisati Einsteinovu jednadžbu gravitacije u "+" konvenciji, tj.
$$ R_{\mu\nu} - \frac{1}{2}g_{\mu\nu}R = + \frac{8\pi G}{c^4} T_{\mu\nu}$$

<p>Riccijev skalar ($R^\mu_{\ \, \mu}$):</p>

In [ ]:
Ricci_scalar = g.ricci_scalar()
Ricci_scalar.display()

(Za razliku od Riccijevog tenzora koji je invarijantan na odabir konvencije za metrički tenzor, Riccijev skalar bi u drugoj konvenciji za metrički tenzor imao obrnuti predznak, npr. na gornjoj stranici wikipedije.) Uočavamo da je za statičko prostorvrijeme Riccijev skalar

In [ ]:
Ricci_scalar.expr().subs({a.expr():1, diff(a.expr(),t):0, diff(a.expr(),t,2):0})

dakle proporcionalan je Gaussovoj zakrivljenosti plohe (relativni minus je u ovoj konvenciji za metriku jer je zakrivljen prostorni dio). To je manifestacija Gaussovog "teorema egregium" po kojem je ekstrinsično svojstvo plohe (radijus zakrivljenosti) potpuno određeno intrinsičnom metrikom na plohi.

<p>4-brzina kozmičkog fluida:</p>

In [ ]:
u = M.vector_field('u')
u[0] = 1/c
u.display()

<p>Tenzor energije-impulsa $T$ za savršeni fluid:</p>

In [ ]:
u_form = u.down(g) # the 1-form associated to u by metric duality
T = (rho*c^2+p)*(u_form*u_form) - p*g
T.set_name('T')
print(T)
T.display()

<p>Trag od $T$:</p>

In [ ]:
Ttrace = g.inverse()['^ab']*T['_ab']
Ttrace.display()

<p>Einsteinova jednadžba gravitacije. Kako je gore navedeno, nužno u "+" konvenciji: $R_{\mu \nu} - {1 \over 2} R g_{\mu \nu} = {8 \pi G} T_{\mu \nu}$</p>

In [ ]:
E1 = Ricci - Ricci_scalar/2*g - (8*pi*G/c^4)*T
print("Friedmannova jednadžba:\n")
(E1[0,0]/3).expr().expand() == 0      # dividing everything by 3

<p>Trace-reversed version of the Einstein equation: $R_{\mu \nu} - \Lambda g_{\mu \nu} = {8 \pi G} \left(T_{\mu \nu} - {1 \over 2}T\,g_{\mu \nu}\right)$</p>

In [ ]:
E2 = Ricci - (8*pi*G/c^4)*(T - Ttrace/2*g)
print("Jednadžba ubrzanja:\n")
(E2[0,0]/3).expr().expand() == 0